# The oldest business in the world
<p><img src="datasets/400px-Eingang_zum_St._Peter_Stiftskeller.jpg" alt="The entrance to St. Peter Stiftskeller, a restaurant in Saltzburg, Austria. The sign over the entrance shows &quot;803&quot; - the year the business opened."></p>
* Disclaimer: Project initial part of the DataCamp's Data Science Projects. It was improved by adding the step 1 with the creation of the database and sql tables.
<p><em>Image: St. Peter Stiftskeller, founded 803. Credit: <a href="https://commons.wikimedia.org/wiki/File:Eingang_zum_St._Peter_Stiftskeller.jpg">Pakeha</a>.</em></p>
<p>An important part of business is planning for the future and ensuring that the company survives changing market conditions. Some businesses do this really well and last for hundreds of years.</p>
<p>BusinessFinancing.co.uk <a href="https://businessfinancing.co.uk/the-oldest-company-in-almost-every-country">researched</a> the oldest company that is still in business in (almost) every country and compiled the results into a dataset. In this project, you'll explore that dataset to see what they found.</p>
<p>The database contains three tables.</p>
<h3 id="categories"><code>categories</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>category_code</code></td>
<td>varchar</td>
<td>Code for the category of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>category</code></td>
<td>varchar</td>
<td>Description of the business category.</td>
</tr>
</tbody>
</table>
<h3 id="countries"><code>countries</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>country_code</code></td>
<td>varchar</td>
<td>ISO 3166-1 3-letter country code.</td>
</tr>
<tr>
<td style="text-align:left;"><code>country</code></td>
<td>varchar</td>
<td>Name of the country.</td>
</tr>
<tr>
<td style="text-align:left;"><code>continent</code></td>
<td>varchar</td>
<td>Name of the continent that the country exists in.</td>
</tr>
</tbody>
</table>
<h3 id="businesses"><code>businesses</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>business</code></td>
<td>varchar</td>
<td>Name of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>year_founded</code></td>
<td>int</td>
<td>Year the business was founded.</td>
</tr>
<tr>
<td style="text-align:left;"><code>category_code</code></td>
<td>varchar</td>
<td>Code for the category of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>country_code</code></td>
<td>char</td>
<td>ISO 3166-1 3-letter country code.</td>
</tr>
</tbody>
</table>


<p> But before we start quering the tables, let's set-up the database with the three tables</p>

### 1. Setting up the database with the 3 tables: categories, countries, businesses

In [5]:
import sqlite3
import pandas as pd

db_name='businessfinancing.db'

db_sql_file='datasets/createdb.sql'

files_tables={
    'categories':'datasets/categories.csv',
    'countries':'datasets/countries.csv', 
    'businesses':'datasets/businesses.csv'
}

def open_connection(db_name):
    conn = sqlite3.connect(db_name)
    return conn

def create_db(conn, db_sql_file, files_tables):
    with open(db_sql_file, 'r') as file:
        sql_script = file.read()

    cur = conn.cursor()
    cur.executescript(sql_script)
    conn.commit()
    print("The tables have been created successfully using the 'createdb.sql' script.")
    
    for table, csv_file in files_tables.items():
        df = pd.read_csv(csv_file)
        df.to_sql(table, conn, if_exists='replace', index=False)
    print("The data has been loaded successfully from the CSV files.")


def run_query(conn, query):
    df = pd.read_sql_query(query, conn)
    return df

def close_connection(conn):
    conn.close()


conn = open_connection(db_name)
create_db(conn, db_sql_file, files_tables)


close_connection(conn)

The tables have been created successfully using the 'createdb.sql' script.
The data has been loaded successfully from the CSV files.


### 2. Select the oldest and newest founding years from the businesses table

<p>Let's begin by looking at the range of the founding years throughout the world.</p>

In [9]:
conn = open_connection(db_name)
query = '''SELECT 
                MIN(year_founded) as oldest_founding_year,
                MAX(year_founded) as newest_founding_year
            FROM businesses'''
df = run_query(conn, query)

display(df)

,oldest_founding_year,newest_founding_year
0,578,1999


### 3. How many businesses were founded before 1000?
<p>Wow! That's a lot of variation between countries. In one country, the oldest business was only founded in 1999. By contrast, the oldest business in the world was founded back in 578. That's pretty incredible that a business has survived for more than a millennium.</p>
<p>I wonder how many other businesses there are like that.</p>

In [14]:
# Getting the count of rows in businesses where the founding year was before 1000

query = '''SELECT count(*) as count_businesses
            FROM businesses
            WHERE year_founded < 1000'''
df = run_query(conn, query)

display(df)

,count_businesses
0,6


### 4. Which businesses were founded before 1000?
<p>Having a count is all very well, but I'd like more detail. Which businesses have been around for more than a millennium?</p>

In [15]:
# Selecting all columns from businesses where the founding year was before 1000. Arranging the results from oldest to newest

query = '''SELECT *
            FROM businesses
            WHERE year_founded < 1000
            ORDER BY year_founded ASC'''
df = run_query(conn, query)

display(df)


,business,year_founded,category_code,country_code
0,Kongō Gumi,578,CAT6,JPN
1,St. Peter Stifts Kulinarium,803,CAT4,AUT
2,Staffelter Hof Winery,862,CAT9,DEU
3,Monnaie de Paris,864,CAT12,FRA
4,The Royal Mint,886,CAT12,GBR
5,Sean's Bar,900,CAT4,IRL


### 5. Exploring the categories
<p>Now we know that the oldest, continuously operating company in the world is called Kongō Gumi. But was does that company do? The category codes in the <code>businesses</code> table aren't very helpful: the descriptions of the categories are stored in the <code>categories</code> table.</p>
<p>This is a common problem: for data storage, it's better to keep different types of data in different tables, but for analysis, you want all the data in one place. To solve this, you'll have to join the two tables together.</p>

In [16]:
# Selecting business name, founding year, and country code from businesses; and category from categories
# where the founding year was before 1000, arranged from oldest to newest

query = '''SELECT
                b.business,
                b.year_founded,
                b.country_code,
                c.category
            FROM 
                businesses b
            JOIN 
                categories c ON b.category_code=c.category_code
            WHERE 
                year_founded < 1000
            ORDER BY 
                year_founded ASC'''

df = run_query(conn, query)

display(df)

,business,year_founded,country_code,category
0,Kongō Gumi,578,JPN,Construction
1,St. Peter Stifts Kulinarium,803,AUT,"Cafés, Restaurants & Bars"
2,Staffelter Hof Winery,862,DEU,"Distillers, Vintners, & Breweries"
3,Monnaie de Paris,864,FRA,Manufacturing & Production
4,The Royal Mint,886,GBR,Manufacturing & Production
5,Sean's Bar,900,IRL,"Cafés, Restaurants & Bars"


### 6. Counting the categories
<p>With that extra detail about the oldest businesses, we can see that Kongō Gumi is a construction company. In that list of six businesses, we also see a café, a winery, and a bar. The two companies recorded as "Manufacturing and Production" are both mints. That is, they produce currency.</p>
<p>I'm curious as to what other industries constitute the oldest companies around the world, and which industries are most common.</p>

In [23]:
# Selecting the category and count of category arranged by descending count, limited to 10 most common categories

query = '''SELECT 
                c.category,
                count(*) as count_category
            FROM 
                categories c
            JOIN 
                businesses b ON b.category_code=c.category_code
            GROUP BY 
                category
            ORDER BY 
                count(*) DESC
            LIMIT 10'''

df = run_query(conn, query)

display(df)

,category,count_category
0,Banking & Finance,37
1,"Distillers, Vintners, & Breweries",22
2,Aviation & Transport,19
3,Postal Service,16
4,Manufacturing & Production,15
5,Media,7
6,Food & Beverages,6
7,"Cafés, Restaurants & Bars",6
8,Agriculture,6
9,Tourism & Hotels,4


### 7. Oldest business by continent
<p>It looks like "Banking &amp; Finance" is the most popular category. Maybe that's where you should aim if you want to start a thousand-year business.</p>
<p>One thing we haven't looked at yet is where in the world these really old businesses are. To answer these questions, we'll need to join the <code>businesses</code> table to the <code>countries</code> table. Let's start by asking how old the oldest business is on each continent.</p>

In [22]:
# Selecting the oldest founding year (as "oldest") from businesses, and continent from countries for each continent, ordered from oldest to newest 

query = '''SELECT
                MIN(b.year_founded) as oldest,
                c.continent
            FROM 
                businesses b
            JOIN 
                countries c ON c.country_code=b.country_code
            GROUP BY 
                c.continent
            ORDER BY 
                MIN(b.year_founded) ASC'''
df = run_query(conn, query)

display(df)

,oldest,continent
0,578,Asia
1,803,Europe
2,1534,North America
3,1565,South America
4,1772,Africa
5,1809,Oceania


### 8. Joining everything for further analysis
<p>Interesting. There's a jump in time from the older businesses in Asia and Europe to the 16th Century oldest businesses in North and South America, then to the 18th and 19th Century oldest businesses in Africa and Oceania. </p>
<p>As mentioned earlier, when analyzing data it's often really helpful to have all the tables you want access to joined together into a single set of results that can be analyzed further. Here, that means we need to join all three tables.</p>

In [21]:
# Selecting the business, founding year, category, country, and continent

query = '''SELECT
                b.business,
                b.year_founded,
                cat.category,
                c.country,
                c.continent
            FROM
                businesses b
            JOIN
                categories cat ON b.category_code=cat.category_code
            JOIN
                countries c ON c.country_code=b.country_code'''
df = run_query(conn, query)

display(df)

,business,year_founded,category,country,continent
0,Hamoud Boualem,1878,Food & Beverages,Algeria,Africa
1,Communauté Électrique du Bénin,1968,Energy,Benin,Africa
2,Botswana Meat Commission,1965,Agriculture,Botswana,Africa
3,Air Burkina,1967,Aviation & Transport,Burkina Faso,Africa
4,Brarudi,1955,"Distillers, Vintners, & Breweries",Burundi,Africa
...,...,...,...,...,...
158,Cafe Brasilero,1877,"Cafés, Restaurants & Bars",Uruguay,South America
159,Hacienda Chuao,1660,Food & Beverages,"Venezuela, Bolivarian Republic of",South America
160,Australia Post,1809,Postal Service,Australia,Oceania
161,Bank of New Zealand,1861,Banking & Finance,New Zealand,Oceania


### 9. Counting categories by continent
<p>Having <code>businesses</code> joined to <code>categories</code> and <code>countries</code> together means we can ask questions about both these things together. For example, which are the most common categories for the oldest businesses on each continent?</p>

In [20]:
# Count the number of businesses in each continent and category

query = '''SELECT
                c.continent,
                cat.category,
                count(*) as counts_businesses
            FROM
                businesses b
            JOIN
                categories cat ON b.category_code=cat.category_code
            JOIN
                countries c ON c.country_code=b.country_code
            GROUP BY
                    cat.category,
                    c.continent
            ORDER BY count(*) DESC'''

df = run_query(conn, query)

display(df)

,continent,category,counts_businesses
0,Africa,Banking & Finance,17
1,Europe,"Distillers, Vintners, & Breweries",12
2,Africa,Aviation & Transport,10
3,Africa,Postal Service,9
4,Europe,Manufacturing & Production,8
5,Asia,Aviation & Transport,7
6,Asia,Banking & Finance,6
7,Europe,Banking & Finance,5
8,North America,"Distillers, Vintners, & Breweries",5
9,North America,Banking & Finance,4


### 10. Filtering counts by continent and category
<p>Combining continent and business category led to a lot of results. It's difficult to see what is important. To trim this down to a manageable size, let's restrict the results to only continent/category pairs with a high count.</p>

In [18]:
# Repeating that previous query, filtering for results having a count greater than 5

query = '''SELECT
                cat.category,
                c.continent,
                count(*) as counts_businesses
            FROM
                businesses b
            JOIN
                categories cat ON b.category_code=cat.category_code
            JOIN
                countries c ON c.country_code=b.country_code
            GROUP BY
                    cat.category,
                    c.continent
            HAVING 
                count(*) > 5
            ORDER by count(*) DESC'''

df = run_query(conn, query)

display(df)
close_connection(conn)

,category,continent,counts_businesses
0,Banking & Finance,Africa,17
1,"Distillers, Vintners, & Breweries",Europe,12
2,Aviation & Transport,Africa,10
3,Postal Service,Africa,9
4,Manufacturing & Production,Europe,8
5,Aviation & Transport,Asia,7
6,Banking & Finance,Asia,6
